<a href="https://colab.research.google.com/github/LarryFeng01/MovieReccommendationSystem/blob/main/data340recommendersystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np 
df1=pd.read_csv('drive/MyDrive/Data Sets/tmdb_5000_credits.csv')
df2=pd.read_csv('drive/MyDrive/Data Sets/tmdb_5000_movies.csv')

In [ ]:
df1= df1.drop(['title'],axis=1)
df1.columns = ['id','cast','crew']
df2 = df2.merge(df1,on='id')

In [ ]:
df2.head(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [ ]:
df2['overview'].head(5)

0    In the 22nd century, a paraplegic Marine is di...
1    Captain Barbossa, long believed to be dead, ha...
2    A cryptic message from Bond’s past sends him o...
3    Following the death of District Attorney Harve...
4    John Carter is a war-weary, former military ca...
Name: overview, dtype: object

Now we will convert each overview into a word vector. We will compute the TF-IDF (Term Frequency-Inverse Document Frequency) vectros for each overview. 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

#remove stop words
tfidf = TfidfVectorizer(stop_words='english')

#replace any null values with an empty string
df2['overview'] = df2['overview'].fillna('')

#construct the matrix
tfidf_matrix = tfidf.fit_transform(df2['overview'])

tfidf_matrix.shape

(4803, 20978)

So, there are around 21,000 unique words in the plot descriptions for the 4,803 movies. Now we will compute the similarity score. We will be using the cosine similarity since we are working with words encoded as vectors.

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

#compute cosine similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

Now we want to make a function that takes in a movie title as the input and outputs a list of movies that are the most similar. First, we will need a reverse mapping of movie titles and DataFrame indices.

In [ ]:
indices = pd.Series(df2.index, index=df2['title']).drop_duplicates()

In [ ]:
def get_recc(title, cosine_sim=cosine_sim):

    #get index of movie that matches input
    idx= indices[title]

    #get pairwise similarity scores of all movies with input
    sim_scores = list(enumerate(cosine_sim[idx]))

    #sort movies based on score
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    #get top 10 scores
    sim_scores = sim_scores[1:11]

    #get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    #return the indices
    return df2['title'].iloc[movie_indices]

In [ ]:
get_recc('Avatar')

3604                       Apollo 18
2130                    The American
634                       The Matrix
1341            The Inhabited Island
529                 Tears of the Sun
1610                           Hanna
311     The Adventures of Pluto Nash
847                         Semi-Pro
775                        Supernova
2628             Blood and Chocolate
Name: title, dtype: object

In [ ]:
get_recc("Pirates of the Caribbean: At World's End")

2542    What's Love Got to Do with It
3095              My Blueberry Nights
2102                  The Descendants
1280                        Disturbia
3632             90 Minutes in Heaven
792                  Just Like Heaven
1709     Space Pirate Captain Harlock
1799                     Original Sin
2652       Bathory: Countess of Blood
4423                   Bang Bang Baby
Name: title, dtype: object

In [ ]:
get_recc('The Dark Knight Rises')

65                              The Dark Knight
299                              Batman Forever
428                              Batman Returns
1359                                     Batman
3854    Batman: The Dark Knight Returns, Part 2
119                               Batman Begins
2507                                  Slow Burn
9            Batman v Superman: Dawn of Justice
1181                                        JFK
210                              Batman & Robin
Name: title, dtype: object

As seen, the recommender can give movies that are a little similar, but most of the titles are not similar from the view of a person that has watched all of the movies. So, we will incoporate more data into the model, such as using actors, director, related genres, and movie plot keywords.

In [ ]:
from ast import literal_eval

features = ['cast', 'crew','keywords', 'genres']
for feature in features:
    df2[feature] = df2[feature].apply(literal_eval)

In [ ]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]

        if len(names) > 3:
            names = names[:3]
        return names

    return []

In [ ]:
df2['director'] = df2['crew'].apply(get_director)

features = ['cast','keywords','genres']
for feature in features:
    df2[feature] = df2[feature].apply(get_list)

In [ ]:
df2[['title','cast','director','keywords','genres']].head(5)

,title,cast,director,keywords,genres
0,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron,"[culture clash, future, space war]","[Action, Adventure, Fantasy]"
1,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski,"[ocean, drug abuse, exotic island]","[Adventure, Fantasy, Action]"
2,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",Sam Mendes,"[spy, based on novel, secret agent]","[Action, Adventure, Crime]"
3,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman]",Christopher Nolan,"[dc comics, crime fighter, terrorist]","[Action, Crime, Drama]"
4,John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton]",Andrew Stanton,"[based on novel, mars, medallion]","[Action, Adventure, Science Fiction]"


Now we will make all the names and keyowrds into lowercase and remove all spaces. 

In [ ]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #check if director exists, if not then return nothing
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [ ]:
features = ['cast','keywords','director','genres']

for feature in features:
    df2[feature] = df2[feature].apply(clean_data)

In [ ]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])
    
df2['soup'] = df2.apply(create_soup, axis=1)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df2['soup'])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [ ]:
df2 = df2.reset_index()

indices = pd.Series(df2.index, index=df2['title'])

In [ ]:
get_recc('Avatar', cosine_sim2)

206                         Clash of the Titans
71        The Mummy: Tomb of the Dragon Emperor
786                           The Monkey King 2
103                   The Sorcerer's Apprentice
131                                     G-Force
215      Fantastic 4: Rise of the Silver Surfer
466                            The Time Machine
715                           The Scorpion King
1      Pirates of the Caribbean: At World's End
5                                  Spider-Man 3
Name: title, dtype: object

In [ ]:
get_recc("Pirates of the Caribbean: At World's End", cosine_sim2)

12            Pirates of the Caribbean: Dead Man's Chest
199    Pirates of the Caribbean: The Curse of the Bla...
13                                       The Lone Ranger
17           Pirates of the Caribbean: On Stranger Tides
71                 The Mummy: Tomb of the Dragon Emperor
786                                    The Monkey King 2
75                                            Waterworld
103                            The Sorcerer's Apprentice
131                                              G-Force
215               Fantastic 4: Rise of the Silver Surfer
Name: title, dtype: object

In [ ]:
get_recc('The Dark Knight Rises', cosine_sim2)

65               The Dark Knight
119                Batman Begins
4638    Amidst the Devil's Wings
1196                The Prestige
3073           Romeo Is Bleeding
3326              Black November
1503                      Takers
1986                      Faster
303                     Catwoman
747               Gangster Squad
Name: title, dtype: object

Now, we see that our recommender system is giving better movie recommendations. So, we can conclude that adding more data and, specifically, more broad information to work with will give better recommendations. Therefore, we could explore more topics to feed as inputs into our system such as the production company, i.e. Marvel or DC. In addition, we can add weights to each subject so that the system can recommend movies with more of the same production company or director.